#### cubie truck cluster for WRF CHEM

This note explains the steps in creating cubietruck cluster for WRF CHEM modeling.

##### Installing debian
- cubietruck is preinstalled with Android as its main os for its inbuilt nand storage of 8 GB harddisk. To chane this into debian os follwoing steps were followed.
-  More number of attempts was made for running debain or ubunut os image in cubietruck, Later found a tweaked image from [here](http://www.igorpecovnik.com/2013/12/24/cubietruck-debian-wheezy-sd-card-image/) aat first sight shows promising for WRF CHEM. 
-  The image from [here](http://cubie.nobrainz.de/downloads/Cubietruck_Debian_2.8_wheezy.zip) was copied into 4GB SD card using this command ```dd if=/home/swl-sacon-dst/Documents/GISE_2013/LAB/Cubietruck/CT_Debian_2.7_wheezy/CT_Debian_2.7_wheezy.raw of=/dev/sdb bs=4096; sync```. The system tool gpart was used for foramting the sd card. 
-  AFtre this wrting sd card was loaded into cubietruck(CT) and booted, by default CT is primed for SD card boot than nand, so it booted and the debian was running from the SD card. To write the SD card os into NAND flash, follwoing attempts werew made
-  First attempt: used the provided script with sd card to flash the nand. Though the script ended without any error but failed to boot without sd card. 
-  Second attempt: based on this [page](http://linux-sunxi.org/Cubieboard/Installing_on_NAND) the SD card os image was copied into nand flash by formating and rsync. This also ends in non bootable nand flash. Followed this [note](http://www.cubieforums.com/index.php?topic=1134.0) also, but no positive output.
-  Third attempt: followed as per the used os image page instruction. It is instructed to flash lubunut os on the nand image using All winner supplied tool LiveSuit, then use the nand_flash.sh script. For this live suit was used based on thsi [page](http://linux-sunxi.org/LiveSuit) but ends in problem of not recognized by laptop for the allwinner USB of cubitruck attached to it. Later it is found that from [here](https://groups.google.com/forum/#!topic/cubieboard/pKFduXRdeNc) it is of problem with kernal version. So started livesuit in a nother ubunut 12.04 with keranl version 3.02, it recongnized cuibitruck usb and flashed its nand with lubuntu os. 
-  In the lubuntu os, then used the sd card debian image and flashed, this time the nand was flashed and cubietruck was running without sd card os, debain wheezy in NAND flash.

#### Compile WRF CHEM
-  Choosing the right gcc and gfortran based on update-alternatives, and it choosed gcc 4.7 and gfortran 4.7. With different gcc and gfrotran version as of GSI compilation it ends in gcc, gfortran testing of wrf compilaiton a erroneaous. Same version of gcc and gfortran cleared the test without any error. 

```bash
sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-4.4 60 --slave /usr/bin/g++ g++ /usr/bin/g++-4.4 
sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-4.6 40 --slave /usr/bin/g++ g++ /usr/bin/g++-4.6 
sudo update-alternatives --config gcc      
sudo update-alternatives --install /usr/bin/gfortran gfortran	/usr/bin/gfortran-4.7 60
sudo update-alternatives --install /usr/bin/gfortran gfortran /usr/bin/gfortran-4.4 10
sudo update-alternatives --config gfortran
```
-  Following GSI compilation Openmpi 1.6.1 was compiled in cubietruck debian wheezy. It was based on [this](http://verahill.blogspot.in/2012/06/compiling-openmpi-on-debian-wheezy.html). Earlier openmpi 1.8.1 compilation ends in failure. 

```bash
wget http://www.open-mpi.org/software/ompi/v1.6/downloads/openmpi-1.6.tar.bz2
tar xvf openmpi-1.6.tar.bz2
cd openmpi-1.6/
sudo mkdir /opt/openmpi/
sudo chown ${USER} /opt/openmpi/
./configure --prefix=/opt/openmpi/1.6/ --with-sge
make
make install
export PATH="$PATH:/home/.openmpi/bin"
export LD_LIBRARY_PATH="$LD_LIBRARY_PATH:/home/.openmpi/lib/"
```
- netcdf compile

```fortran
export DIR=/home/wrf_libs
export CC=gcc
export CXX=g++
export FC=gfortran
export FCFLAGS=-g
export F77=gfortran
export FFLAGS=-g
export CXXFLAGS=-g
export PATH=$DIR/netcdf363/bin:$PATH
export NETCDF=$DIR/netcdf363
mpif90 -c 02_fortran+c+netcdf+mpi_f.f
mpicc -c 02_fortran+c+netcdf+mpi_c.c
mpif90 02_fortran+c+netcdf+mpi_f.o \
02_fortran+c+netcdf+mpi_c.o \
     -L${NETCDF}/lib -lnetcdf
mpirun ./a.out
```
- wrfchem compile
- doing NAND flash on CT sh nand-install.sh
```bash                                             
 #    #   ##   #####  #    # # #    #  ####  
 #    #  #  #  #    # ##   # # ##   # #    # 
 #    # #    # #    # # #  # # # #  # #      
 # ## # ###### #####  #  # # # #  # # #  ### 
 ##  ## #    # #   #  #   ## # #   ## #    # 
 #    # #    # #    # #    # # #    #  ####  
This script will NUKE / erase your NAND partition and copy content of SD card to it
Proceed (y/n)? (default: y): 
nand-install.sh: 45: [: unexpected operator
Formatting and optimizing NAND rootfs ... up to 30 sec
mke2fs 1.42.5 (29-Jul-2012)
e2fsck 1.42.5 (29-Jul-2012)
Pass 1: Checking inodes, blocks, and sizes
Pass 2: Checking directory structure
Pass 3: Checking directory connectivity
Pass 4: Checking reference counts
Pass 5: Checking group summary information
/dev/nand2: 11/468640 files (0.0% non-contiguous), 66302/1871872 blocks
Creating NAND bootfs ... few seconds
Creating NAND rootfs ... up to 5 min
All done. Press a key to power off, then remove SD to boot from NAND
```